## MR Semantics

Let's start with some definitions:

#### Shuffle

This is the routing process of mapper outputs to reducer inputs.

<img src="images/hadoopsem.png" width=512 />

#### Partition

* Partition is the output file of a reducer _process_ (not a single reducer).
  * Contains many reducer keys
 
#### Combiner

This image is linked from https://data-flair.training/blogs/hadoop-combiner-tutorial/.  Please refer to their page.

<img src="images/combiner.jpeg" width=512 />

Combiner is a function that runs on the outputs of the mapper before the shuffle.

* Combiner executes on the mappers $ \langle key,value \rangle$ output while in memory at the mapper
  * It is possible to write unique combiner and reduce classes
  * It is common to use the reducer as a combiner
  * Combiner must have algebraic propertics, i.e. `reduce(combine(A),combine(B)) == reduce(A,B)` 
  * Traditional reduce operators (aggregates and extrema) work in combiners
* Combiner in WordCount:
  * compute sum output by mapper for each key and send a single aggregated value to reducers
* Combiner for Maximum: 
  * compute maximum value for each key.  
  * Reducer computes a maximum of maxima.


### The Map/Reduce Sorting Guarantee

* Map: extracts a sorting key from the value
$$
 \langle key, value \rangle -> \langle sort\_key, output\_value \rangle
$$
* Shuffle does not sort strictly:
  * it route's to reducer based on sort key.
  * typically hashing maps key to reducer
  * Keys are sorted as they are presented to the reducer

Here is the guarantee:

> We guarantee that within a given partition, the intermediate key/value pairs are processed in increasing key order. This ordering guarantee makes it easy to generate
a sorted output file per partition, which is useful when
the output file format needs to support efficient random
access lookups by key, or users of the output find it convenient to have the data sorted._

The ordering guarantees sort within partitions
* To sort completely:
  * All output to a single partition (use one reducer)
  * Customize the shuffle function (quite complex and can introduce skew)
  * The default shuffle uses hashing (for load balance)

* The Google paper optimizes sort by customizing shuffle so that partitions are ordered, not randomized
  * Run a M/R job to learn the key distribution
  * Specify a shuffle based on evenly paritioning the key distribution
  * This is also how Hadoop!’s sort record worked